# PDF 文書の内容にもとづいて回答するボットの作り方を理解する
## はじめに
生成 AI や大規模言語モデル （LLM）は、膨大な量のテキストデータのトレーニングを受けており、テキストを生成し、言語を翻訳し、さまざまな種類のクリエイティブ コンテンツを作成し、有益な方法で質問に答えることができます。ただし、LLM には、企業で使用する場合の問題点がいくつかあります。
その一つに、生成 AI は誤った情報を含むテキストを生成したり、誤った情報を翻訳したりする可能性があります。企業における生成 AI 利用では、これが問題となることがあります。

そこで生成 AI をエンジンとして利用しながら、企業内の信頼のおける情報の中から適切な回答してほしいというユースケースがあります。

これを実現する大きな処理の流れは次のようになっています。

1.   PDF ファイルを読み込む
2.   ページごとでもよいのですが、もう少し細かい単位にテキストを分割する
3.   分割したテキスト情報を PaLM API を利用してベクトル化 / エンべディング
4.   ベクトル情報とテキスト情報の関連を保持する
5.   ユーザーの問い合わせ内容をベクトル化 / エンべディングする
6.   5. のベクトルと最も類似度の高いベクトルを複数個取得する
7.   6. のベクトルを生成元のテキスト情報を取得する
8.   複数個のテキスト情報をコンテキスト情報として与え、この情報の中から質問の回答を作成するように LLM に問い合わせをする
9.   PDF 文書の内容にもとづく回答が得られます

これを実現するために次の技術要素が必要となりますが、Google Cloud ではそれぞれに対応するソリューションを提供しています。
 - エンべディング --> Vertex AI Embeddings for Text
 - ベクトル検索 --> Vertex AI Vector Search ( AlloyDB pgvector / CloudSQL pgvector でも実現可能)
 - 最終回答の生成 --> Vertex AI PalM API

また、上記の処理フローをフルスクラッチで開発するとそれなりに工数が必要ですが、生成 AI 利用時のフレームワークである LangChain を利用すると簡単に実現できてしまいます。コーディング上は 3. - 4. で1行、5. - 8.が1行で済んでしまうのは開発者にとってはありがたいです。LangChain 様々です。

この処理を、Google Cloud の Vertex AI PaLM API を利用して確認します。

## 環境セットアップ

python のバージョンを確認します。最新の LangChain は `Requires-Python >=3.8.1,<4.0`が前提となっています。

In [ ]:
import sys
print(sys.version)

前提パッケージを導入します。

In [ ]:
# Install Vertex AI LLM SDK
! pip install google-cloud-aiplatform langchain pypdf PyCryptodome chromadb --upgrade --user

**※ 注意: ここでカーネルを再起動します。**

* Colab の場合、上記のログに"RESTART RUNTIME"ボタンをが表示された場合、ボタンを押してカーネルをリスタートします。
* Vertex AI Workbench の場合、メニューよりカーネルのリスタートを実行します。

続いて、Google Cloud でプロジェクトを作成し Vertex AI API を有効化します。

また、このコードを実行するユーザーに`Vertex AI ユーザー`のロールを付与します。

Colab の場合、以下を実行し Vertex AI API のユーザー権限をもつアカウントでログインします。 Vertex AI Workbench の場合はスキップされます。

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

環境変数などを定義します。 Google Cloud のプロジェクト ID を指定してください。

In [ ]:
PROJECT_ID = "<your_project_id>"  # @param {type:"string"}
REGION = "us-central1"

Vertex AI と LangChain のライブラリーの導入を確認します。 LangChain v0.0.208 で動作確認しています。

In [ ]:
import langchain
from google.cloud import aiplatform

print(f"LangChain version: {langchain.__version__}")
print(f"Vertex AI SDK version: {aiplatform.__version__}")

import vertexai
vertexai.init(project=PROJECT_ID, location=REGION)

## Vertex AI PaLM API の準備

LangChain を利用して PaLM API のText、Chat、Embeddings モデルを取得します。

In [ ]:
from langchain.llms import VertexAI
from langchain.chat_models import ChatVertexAI
from langchain.embeddings import VertexAIEmbeddings

# Text model instance integrated with LangChain
llm = VertexAI(
    model_name="text-bison",
    max_output_tokens=512,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

# Chat instance integrated with LangChain
chat = ChatVertexAI()

# Embeddings API integrated with LangChain
embedding = VertexAIEmbeddings(model_name="textembedding-gecko-multilingual")

## PDF の読み込みとベクトル化

PDF ファイルを読み込みます。

In [ ]:
# Ingest PDF files
from langchain.document_loaders import PyPDFLoader

# set PDF urls
urls = [
    # Google Cloud セキュリティ ホワイトペーパー
    #"https://services.google.com/fh/files/misc/security_whitepapers_4_booklet_jp.pdf", 
    # 「Google Cloud Day: Digital '22 - 15 のトピックから学ぶ」🌟eBook
    "https://lp.cloudplatformonline.com/rs/808-GJW-314/images/Google_ebooks_all_0614.pdf",
]
documents = []
for url in urls:
    documents += PyPDFLoader(url).load()

print(f"# of documents = {len(documents)}")

PDF から抽出した 1 ページごとのテキスト情報を一度統合し、少しのオーバーラップを含むより小さなチャンクに分割します。PDF 文書の 1 ページの内容が多くなかったり、ページごとに内容が分かれている場合、このステップは省略可能です。

In [ ]:
# split the documents into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
content = "\n\n".join(doc.page_content for doc in documents)
texts=text_splitter.split_text(content)
print(f"# of texts = {len(texts)}")

抽出したテキスト情報をエンベディングと呼ばれるベクトル情報として保管します。
ここではベクトル情報の保存検索にインメモリで動作する軽量な Chroma を利用します。

大規模なベクターストアとして、Google Cloud では [Vertex AI Vector Search](https://cloud.google.com/vertex-ai/docs/vector-search/overview?hl=ja) の利用を推奨しています。Vertex AI Vector Search は、拡張性が高くレイテンシが低いベクトル類似性マッチング（近似最近傍探索）サービスを提供します。

また、エンベディングに PaLM API の textembedding-gecko を利用します。

In [ ]:
# Store docs in local vectorstore as index
# it may take a while since API is rate limited
from langchain.vectorstores import Chroma

db = Chroma.from_texts(texts, embedding)

In [ ]:
# Expose index to the retriever
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

## PDF の内容で Q/A をする

ユーザーが質問した質問に対して、ベクトル検索で取得した類似のテキスト情報の中から回答を出すようにします。この仕組みを簡単に実現するフレームワークとして LangChain の RetrievalQA を利用します。

In [ ]:
# Create chain to answer questions
from langchain.chains import RetrievalQA

# Uses LLM to synthesize results from the search index.
# We use Vertex PaLM Text API for LLM
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True
)

ここで質問を定義します。

In [ ]:
#query = input("Enter query:")
query = "Cloud Spanner の特徴を教えて下さい。300字程度で回答してください。" # @param {type:"string"}


PaLM API for text に直接問い合わせをした場合は、一般の知識に基づいて回答します。その内容を確認します。

In [ ]:
llm(query)

PDF 情報からの回答はどのようになるでしょう？

In [ ]:
response = qa({"query": query})
response["result"]

回答内容を作成したソースを確認します。

In [ ]:
response["source_documents"]

どうして PDF の内容をもとに回答しているかを理解するには、上で利用した LangChain の RetrievalQA の `stuff` タイプのソースコードを参照するのが分かりやすいです。

https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/chains/question_answering/stuff_prompt.py

```
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:"""
```
```
次のコンテキストをもとに、最後の質問に答えてください。もし答えが分からない場合は、分からないと答えてください。勝手に答えをでっち上げないでください。
```

上記のように、最終の回答はプロンプト エンジニアリングで、正確な情報ソースをプロンプトにコンテキスト情報として渡し、そのコンテキスト情報の範囲内で回答するように LLM に依頼しています。
以上、ありがとうございました。

## 参考情報
- [Getting Started with LangChain 🦜️🔗 + Vertex AI PaLM API](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/orchestration/langchain/intro_langchain_palm_api.ipynb)
- [Question Answering with Large Documents using LangChain 🦜🔗](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/use-cases/document-qa/question_answering_documents_langchain.ipynb)
- [LangChain](https://python.langchain.com/docs/get_started/introduction.html)
- [Overview of Generative AI on Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview)
